# Exploring Kickstarter Project Data
## by Michael Mosin

## Preliminary Wrangling

This document explores a dataset comprised of various attributes for an assortment of 3786 Kickstarter projects

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [ ]:
# Import dataset
# Dataset downloaded from CSV link under "2019-05-16" on site: https://webrobots.io/kickstarter-datasets/
df = pd.read_csv('Kickstarter.csv')

In [ ]:
# Adding ability to view all dataframe columns
# as per https://stackoverflow.com/questions/49188960/how-to-show-all-of-columns-name-on-pandas-dataframe/49189503
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.info()

In [ ]:
df.duplicated().value_counts()

In [ ]:
# Make copy of main dataframe so as to keep original data intact.
df_copy = df.copy()
df_copy.shape

## Tracking Data Quality and Tidiness Issues:

### Quality:

- Variables "created_at", "deadline", "launched_at", and "state_changed_at" are set in unix time instead of readable datetime
- Variables with financial values such as "converted_pledged_amount", "goal", "pledged", and "usd_pledged" are set to different decimal places, and should be rounded to at most two decimal places
- Variables "friends", "is_backing", "is_starred", and "permissions" only have one entry and should be dropped
- Only two entries are missing data for "location" (not a big deal, given that we have "country" data; these)
- Only eleven entries are missing data for "usd_type" (this variable is not important to the investigation)

### Tidiness:

- Data entries in the columns "category", "creator", "location", "photo", "profile", and "urls" contain multiple pieces of information. If separated, they could be their own dataframes or made into additional columns in the main dataframe.
    - The "category" variable can garner category and sub-category info for the projects
    - The "location" variable can garner data regarding the project's state name, city name, and city type
    - The "creator","photo", "profile", "urls" variables have no data that is relevant to this project and should be dropped


## Addressing Data Quality and Tidiness Issues

### Quality: 

#### Remove (essentially) empty columns

In [ ]:
# Remove variables that only have one entry.
df_copy = df_copy.drop(columns = ["friends", "is_backing", "is_starred", "permissions"])
df_copy.shape

#### Fix time categories

In [ ]:
# Converting unix time to readable date-time
# as per https://stackoverflow.com/questions/19231871/convert-unix-time-to-readable-date-in-pandas-dataframe
date_cols = ["created_at", "deadline", "launched_at", "state_changed_at"]
for i in date_cols:
    df_copy[i] = pd.to_datetime(df_copy[i],unit='s')

df_copy[date_cols].head()

In [ ]:
df_copy[date_cols].describe()

#### Fix financial categories

In [ ]:
# Round financial values to at most two decimal places
money_cols = ["converted_pledged_amount", "goal", "pledged", "usd_pledged"]
for i in money_cols:
    df_copy = df_copy.round(2)

df_copy[money_cols].head()

### Tidiness: 

#### Feature Engineering - address tidiness issue of "category" variable by creating variables holding extracted values for main category and sub-category:

In [ ]:
# View full of string entries for "category" variable to gauge the complexity of category strings:
df_copy['category'][15]

In [ ]:
df_copy['category'][798]

In [ ]:
# Extract product categories and sub-catgories from strings in "category" variable into their own columns in dataframe
# (Used regular expression)

import re  

df_copy['main_cat'] = ''
df_copy['sub_cat'] = ''

for i in np.arange(df_copy.shape[0]):
    match = re.findall('(([- &\'\\\\]|\w+)+)', df_copy['category'][i])
    df_copy['main_cat'][i] = match[5][0].title()
    df_copy['sub_cat'][i] = match[3][0].replace('\\','')

In [ ]:
df_copy[['name','main_cat', 'sub_cat', 'category']].head() 

In [ ]:
df_copy.main_cat.value_counts()

In [ ]:
df_copy.sub_cat.value_counts()

#### Feature Engineering - address tidiness issue of "location" variable by creating variables holding extracted values for state, city, and type of city:

In [ ]:
df_copy[df_copy.location.isnull()]

In [ ]:
# View full string entry for "location" variable of third row entry to gauge the complexity of location strings:
df_copy['location'][2]

In [ ]:
# Extract product (country) states, cities, and city types from strings in "location" variable into their own columns in dataframe
# (Used regular expression)

df_copy['location_state'] = ''
df_copy['location_city'] = ''
df_copy['location_type'] = ''

for i in np.arange(df_copy.shape[0]):
    if pd.notna(df_copy.location[i]) == True:
        match = re.findall('((?:[^"]\w+)+)', df_copy['location'][i])
        df_copy['location_state'][i] = match[17]
        df_copy['location_city'][i] = match[3]
        df_copy['location_type'][i] = match[19]
    else:
        df_copy['location_state'][i] = 'NaN'
        df_copy['location_city'][i] = 'NaN'
        df_copy['location_type'][i] = 'NaN'
    

In [ ]:
df_copy[['name', 'country', 'location_state', 'location_city', 'location_type', 'location']][1930:1933]

### What is the structure of your dataset?

There are 3786 Kickstarter projects in this dataset, with a total of 37 features, some of which are untidy, and some of which are not of interest for my exploration. I have engineered a few categorical features (related to project ctegories and location) which may come to be useful for exploration. 


### What is/are the main feature(s) of interest in your dataset?

I am interested in finding out which project qualities correlate with different types of project outcomes (or, the final "state" of the project). 


### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

I believe the following features could illuminate patterns in project outcomes:
- Number of backers
- Length of time project was open
- Total money pledged (relative to funding goal)
- Project category
- Project location (country, city type)
- If project had "spotlight"
- If project was "staff pick"

In [ ]:
# Save wrangled dataframe to new CSV file to make future manipulating easier
df_copy.to_csv('data_wrangled.csv', index=False)

## Streamlining Wrangled Dataset 

### Removing extra variables, and engineering other potentially relevant features

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [ ]:
# Import wrangled dataset
df = pd.read_csv('data_wrangled.csv')
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
# Remove unnecessary variables
df.drop(columns = ['category',
                   'converted_pledged_amount',
                   'creator',
                   'currency_symbol',
                   'currency_trailing_code',
                   'current_currency',
                   'disable_communication',
                   'fx_rate',
                   'location',
                   'photo',
                   'profile',
                   'slug',
                   'source_url',
                   'static_usd_rate',
                   'urls',
                   'usd_type'],
       inplace = True)
df.head()

#### Engineering features related to the time variables:

- created_at
- launched_at
- deadline
- state_changed_at

Reference:

http://www.datasciencemadesimple.com/difference-two-timestamps-seconds-minutes-hours-pandas-python-2/
https://docs.scipy.org/doc/numpy/reference/arrays.datetime.html

In [ ]:
# Confirm that time variables are of 'datetime64' type:
date_cols = ["created_at", "deadline", "launched_at", "state_changed_at"]
for i in date_cols:
    df[i] = pd.to_datetime(df[i])

df[date_cols].dtypes

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

In [ ]:
# Calculate number of days it took to launch project: 'days_to_launch'
# (days between project creation and project launch: 'launched_at' - 'created_at')

df['days_to_launch'] = df['launched_at'] - df['created_at']
df['days_to_launch']=df['days_to_launch']/np.timedelta64(1,'D')
df.days_to_launch.head()

In [ ]:
# Calculate number of days given for project to succeed: 'days_to_succeed'
# (days between project launch and project deadline: 'deadline' - 'launched_at')

df['days_to_succeed'] = df['deadline'] - df['launched_at']
df['days_to_succeed']=df['days_to_succeed']/np.timedelta64(1,'D')
df.days_to_succeed.head()

In [ ]:
# Calculate number of days project was active (or reached its final 'state') : 'days_active'
# (days between project launch and project deadline: 'state_changed_at' - 'launched_at')

df['days_active'] = df['state_changed_at'] - df['launched_at']
df['days_active']=df['days_active']/np.timedelta64(1,'D')
df.days_active.head()

In [ ]:
# Confirm whether final 'state' occurred before or after 'deadline': 'ended_early'
# Faster code instead of for loops as per reference:
# https://stackoverflow.com/questions/27041724/using-conditional-to-generate-new-column-in-pandas-dataframe)

df['ended_early'] = np.where(df.days_active < df.days_to_succeed, True, False)

In [ ]:
df[['days_to_succeed','days_active','ended_early']].head()

#### Engineering Feature: proportion of project funding relative to goal

In [ ]:
# Finding ratio of confirmed funds relative to funding goals: 'funded_prop'
# ('pledged' / 'goal')

df['funded_prop'] = df['pledged'] / df['goal']
df.funded_prop.head()

#### Save cleaner dataframe to new CSV file to make future manipulating easier

In [ ]:
df.to_csv('data_cleaner.csv', index=False)

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [ ]:
# Import wrangled dataset
df = pd.read_csv('data_cleaner.csv')
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
# Set color for charts:
base_color = sb.color_palette()[0]

### What's the distribution of project campaign 'states'?

In [ ]:
sb.countplot(data = df, x = 'state', color = base_color);

#### Looks like more projects succeeded than not.

### What's the proportion of campaigns that ended early?
Plot 'ended_early' counts with proportion percentages over bars

Reference: https://stackoverflow.com/questions/31749448/how-to-add-percentages-on-top-of-bars-in-seaborn

In [ ]:
# create the plot
ax = sb.countplot(data = df, x = 'ended_early', color = base_color)

# add annotations
n_points = df.shape[0]
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 25,
            '{:0.1f}%'.format(100*height/n_points),
            ha = 'center')

plt.show()

Plot 'ended_early' counts with proportion percentages over bars WHILE excluding 'live' projects - since they are still in progress and are skewing the chart.


In [ ]:
# Isolate sub-dataset which excludes projects that are 'live':
df_notlive = df[df['state']!='live']
df_notlive.shape

In [ ]:
# create the plot
ax = sb.countplot(data = df_notlive, x = 'ended_early', color = base_color)

# add annotations
n_points = df_notlive.shape[0]
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 25,
            '{:0.1f}%'.format(100*height/n_points),
            ha = 'center')

plt.show()

#### A follow-up bivariate exploration would be to see whether the different campaign 'states' have a split within them regarding if they ended early.

### What's the distribution of the number of backers per project?

In [ ]:
binsize = 50
bins = np.arange(0, df_notlive['backers_count'].max()+binsize, binsize)

plt.figure(figsize=[8, 5])
plt.hist(data = df_notlive, x = 'backers_count', bins = bins)
plt.xlabel('Number of Backers')
plt.show()

#### Looks like there is a VERY wide distribution of backers, skewed by many projects having zero or few backers, and a few outliers projects with thousands of backers.

In [ ]:
# Plot countplot that illustrates how many projects have fewer than 10 backers:

plt.figure(figsize=[8, 5])
ax = sb.countplot(data = df_notlive, x = 'backers_count', color = base_color)
plt.xlabel('Number of Backers')
plt.ylabel('Number of Projects')
plt.xlim(-0.5,9.5)
plt.ylim(0,350)

# add annotations
for p in ax.patches[0:10]:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:.0f}'.format(height),
            ha = 'center')

plt.show()

#### Since there's a long tail in the distribution, for backer numbers of 10 or greater we plot a histplot with a log transformation: 

In [ ]:
log_binsize = 0.05
bins = 10 ** np.arange(1, np.log10(df_notlive['backers_count'].max())+log_binsize, log_binsize)

plt.figure(figsize=[8, 5])
plt.hist(data = df_notlive, x = 'backers_count', bins = bins)
plt.xscale('log')
plt.xticks([10, 25, 50, 100, 300, 1e3, 2e3, 5e3, 1e4, 2e4, 4e4], [10, 25, 50, 100, 300, '1k', '2k', '5k', '10k', '20k', '40k'])
plt.xlabel('Number of Backers')
plt.show()

#### There still appears to be a long tail, but at least it is a little more constrained under the log transformation.

#### A follow-up bivariate exploration would be to compare the number of backers relative to successful and unsuccessful projects.

### What's the distribution of location type?

In [ ]:
sb.countplot(data = df_notlive, y = 'location_type', color = base_color);

#### Looks like I extracted that information incorrectly. However, even if I had, it's almost useless given the fairly little diversity in distribution of the location type, with the vast majority of projects being in towns. At least we've learned that much.

### What's the distribution of location country?

In [ ]:
sb.countplot(data = df_notlive, y = 'country', color = base_color, order = df_notlive.country.value_counts().index);

#### The vast majority of project campaigns are based in the US, followed by other English-speaking countries. There is some visible representation from Northern Europe, Mexico, Western Europe, and Hong Kong.

### What's the distribution of proportion of funding?

In [ ]:
df_notlive.funded_prop.describe()

In [ ]:
sb.boxplot(data = df_notlive, x = 'funded_prop', color = base_color);

In [ ]:
df_notlive[df_notlive.funded_prop == 27588.23]

#### Looks like there is one data entry (index # 661) which needs to be dropped: the project's funding proportion was severly greater than that of any other, a result of the dataset stating that the campaign funding goal was ONE DOLLAR. This data entry also skews the proportion of successful projects, given that it appears to have met its (unrealistic goal)

In [ ]:
df_notlive = df_notlive.drop([661])
df_notlive[df_notlive.funded_prop == 27588.23]

In [ ]:
sb.boxplot(data = df_notlive, x = 'funded_prop', color = base_color);

In [ ]:
binsize = 0.1
bins = np.arange(0, 5+binsize, binsize)

plt.figure(figsize=[8, 5])
plt.hist(data = df_notlive, x = 'funded_prop', bins = bins)
plt.xlabel('Final Funding as Proportion of Goal')
plt.show()

In [ ]:
log_binsize = 0.01
bins = 10 ** np.arange(-3, np.log10(df_notlive['funded_prop'].max())+log_binsize, log_binsize)

plt.figure(figsize=[15, 5])
plt.hist(data = df_notlive, x = 'funded_prop', bins = bins)
plt.xscale('log')
plt.xticks([.001, .003, .01, .03, .1, .3, 1, 3, 10, 25, 50, 100, 200, 400], [.001, .003, .01, .03, .1, .3, 1, 3, 10, 25, 50, 100, 200, 400])
plt.xlabel('Final Funding as Proportion of Goal')
plt.show()

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Check distribution of 'state' relative to whether project has ended early:

In [ ]:
# Reference:
# https://stackoverflow.com/questions/33271098/python-get-a-frequency-count-based-on-two-columns-variables-in-pandas-datafra
df.groupby(['state','ended_early']).size()

In [ ]:
sb.countplot(data = df, x = 'state', hue = 'ended_early', palette = 'Blues');

#### Since each project 'state' only has projects that EITHER ended early or didn't, there is no distribution of endings to be illustrated within each 'state'. We have learned and confirmed that, not surprisingly, all projects which were 'cancelled' or 'suspended' ended early, and all project campaigns that 'failed' or were 'successful' in reaching their funding goal were open until the end of their deadline.

#### Since I care about whether projects are successful or not, and 'live' project campaigns are still in progress and have yet to be cancelled or suspended, I will remove the project rows that have the state of 'live'.

### Check distribution of 'is_starrable' relative to project 'state':

In [ ]:
df.groupby(['state','is_starrable']).size()

#### Since 'is_starrable' is only true for projects that are 'live', and I am planning on excluding the 'live' projects, we can also remove the 'is_starrable' feature.

### Check distribution of success of projects split by 'main_cat'

In [ ]:
df_notlive['successful'] = np.where(df_notlive.state == 'successful', True, False)

In [ ]:
df_notlive[['name','state','successful']].head()

In [ ]:
plt.figure(figsize=[15, 5])
ax = sb.countplot(data = df_notlive, x = 'main_cat', hue = 'successful', order= df_notlive['main_cat'].value_counts().index);
loc, labels = plt.xticks()
ax.set_xticklabels(labels, rotation=45);

# add annotations
n_points = df_notlive['main_cat'].value_counts()
i=0
for p in ax.patches:
    if i >= 15:
        i = 0
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 5,
                '{:.2f}'.format(height/n_points[i]),
                ha = 'center')
        i += 1
    else:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 5,
                '{:.2f}'.format(height/n_points[i]),
                ha = 'center')
        i +=1
plt.show;

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!